In [1]:
from dataset import import_dataset
from search import dcs,edit_distance
from Oracle import Oracle
import pickle
import pandas as pd
import random
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

### Dataset

In [3]:
# Dataset name
#data_name = "OHSU"
#data_name = "Peking_1"
#data_name = "KKI"
#data_name = "autism"
#data_name = "bip80"
#data_name = "bipolar90"
#data_name = "adhd_90"
data_name = "adhdm_90"

# Black-box
training = True
method = "SF"

In [4]:
# Import data
# Set the right folder !!!
data = import_dataset(data_name,folder="../data/")

Avg Nodes:  116.0
AvgEdges:  667.0


### Classifier

In [6]:
oracle = Oracle(method,data,idc=2)

In [7]:
oracle.train()

[(0.72, 0), (0.7083333333333334, 3), (0.6666666666666666, 4), (0.64, 1), (0.56, 2)]
{0: [KNeighborsClassifier(n_neighbors=23), SVC(kernel='linear')], 1: [KNeighborsClassifier(n_neighbors=2), SVC(kernel='linear')], 2: [KNeighborsClassifier(n_neighbors=9), SVC(kernel='linear')], 3: [KNeighborsClassifier(n_neighbors=10), SVC(kernel='linear')], 4: [KNeighborsClassifier(n_neighbors=1), SVC(kernel='linear')]}
KNeighborsClassifier(n_neighbors=1)


In [8]:
# Evaluate
oracle.evaluate_classifier(data)

Results:
- 58 TP;
- 57 TN;
- 5 FP;
- 3 FN.
Accuracy = 0.9349593495934959
F1 =  0.9354838709677418


### Apply Density-based Counterfactual Search with Clique

In [14]:
%%time
new_info = []
ggc = []
sorted_method = "region"
#sorted_method = "triangle"
l_search_type = "LO"
h_search_type = "HC"
#sorted_method = "random"
ijk = 0
for g,y in data:
    print(ijk,end="-")
    info_g,iter_dcs = dcs(g,y,oracle,sorted_method=sorted_method,
                          g_for_low = l_search_type, g_for_high = h_search_type,
                          dl = True, dh = False, k=2,
                          b_max = 10,max_iter = 200,verbose = False)

    g_c_i = info_g[-1][-1]
    y_hat,y_c_hat = oracle.predict(g),oracle.predict(g_c_i)
          
    new_info.append((g,g_c_i,y,y_hat,y_c_hat,iter_dcs,edit_distance(g,g_c_i)))
    
    if y_hat!=y_c_hat:
        g_cf = info_g[-1][-1]
        ggc.append((g,info_g[-1][-1],y,oracle.predict(g),oracle.predict(g_cf)))
    ijk+=1

0-1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21-22-23-24-25-26-27-28-29-30-31-32-33-34-35-36-37-38-39-40-41-42-43-44-45-46-47-48-49-50-51-52-53-54-55-56-57-58-59-60-61-62-63-64-65-66-67-68-69-70-71-72-73-74-75-76-77-78-79-80-81-82-83-84-85-86-87-88-89-90-91-92-93-94-95-96-97-98-99-100-101-102-103-104-105-106-107-108-109-110-111-112-113-114-115-116-117-118-119-120-121-122-CPU times: user 4min 47s, sys: 6min 2s, total: 10min 49s
Wall time: 3min 23s


In [15]:
#g,g_c_i,y_hat,y_c,iter_i,ed_i
c_cg_found = []
df_l = []
i = 0
for info_i in new_info:
    g, g_c_i, y, y_hat, y_c, iter_i, ed_i = info_i
    # is it a counterfactual?
    df_l.append([i,"All",y_hat,iter_i,ed_i])
    if y_hat!=y_c:
        v_c = True
        df_l.append([i,"C-found",y_hat,iter_i,ed_i])
        c_cg_found.append((i,g,g_c_i,y_hat,"RCLI"))
    else:
        v_c = False
    # is it well-classified?
    if y_hat==y:
        c_ok = "Good"
    else:
        c_ok = "Miss"
    df_l.append([i,c_ok,y_hat,iter_i,ed_i])
    i += 1

In [16]:
# Save results
df = pd.DataFrame(df_l, columns=['g_id','Status','Predicted Class','Iter','Density'])
COUNT_SEARCH = "rdcg_{}_{}".format(l_search_type,h_search_type)
df.to_pickle("results/"+"{}_{}_{}.pickle".format(data_name,method,COUNT_SEARCH))

In [17]:
COUNT_SEARCH = "RCLI"
df = pd.DataFrame(c_cg_found, columns=['gid','G','C','y','Method'])
df.to_pickle("CG/"+"{}_{}_{}_cg.pickle".format(data_name,method,COUNT_SEARCH))

### Apply Backward search (BW)

In [17]:
def edit_distance(g_1,g_2):
    edg1 = set(g_1.edges())
    edg2 = set(g_2.edges())
    return round(len(edg1.symmetric_difference(edg2))/len(edg1.union(edg2)),3)
def edit_distance_raw(g_1,g_2):
    edg1 = set(g_1.edges())
    edg2 = set(g_2.edges())
    return len(edg1.symmetric_difference(edg2))

In [18]:
def less_used_edges(g1,g2):
    """
    """
    edg1 = set(g1.edges())
    edg2 = set(g2.edges())
    sd = list(edg1.symmetric_difference(edg2))
    de = {}
    for i,j in sd:
        de.setdefault(i,0)
        de[i] += 1
        de.setdefault(j,0)
        de[j] += 1
    score_edges = [((i,j),de[i]+de[j]) for i, j in sd]
    score_edges.sort(key=lambda y:y[1])
    return score_edges,de

In [19]:
def edge_backward(g,gc):
    """Edge level score
    """
    score_edges, de = less_used_edges(g,gc)
    gci = gc.copy()
    y_hat = oracle.predict(g)
    yci = oracle.predict(gci)
    ider_count = 1
    #print("original {}, count_i {}".format(y_hat,yci))
    i = 0
    gci_new = gci.copy()
    for e,scorei in score_edges:
        if gci.has_edge(*e):
            gci_new.remove_edge(*e)
            #print("Removed ",e)
        else:
            gci_new.add_edge(*e)
            #print("Add ",e)
        yci = oracle.predict(gci_new)
        ider_count+=1
        #print("original {}, count_i {}".format(y_hat,yci))
        if y_hat==yci:
            #print("Blocked ", i)
            return gci,ider_count
        gci = gci_new.copy()
        i += 1
    #print("Not blocked ", i)
    return gci,ider_count

In [20]:
bw_info = []
bw_graphs = []
new_info_back = []
for new_info_i in new_info:
    print(len(new_info_back),end='-')
    g,gc,y,y_hat,y_c,old_iter,old_ed = new_info_i
    old_ed_raw = edit_distance_raw(g,gc)
    if y_hat!=y_c:
        gccc,new_iter = edge_backward(g,gc)
    else:
        gccc = gc.copy()
        new_iter = 0
    new_ed = edit_distance_raw(g,gccc)
    bw_info.append((old_ed-new_ed,old_ed,y_hat,y_c,edit_distance(g,gccc)))
    bw_graphs.append((g,gccc,y_hat))
    new_info_back.append((g,gccc,y,oracle.predict(g),oracle.predict(gccc),old_iter+new_iter,edit_distance(g,gccc)))

0-1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21-22-23-24-25-26-27-28-29-30-31-32-33-34-35-36-37-38-39-40-41-42-43-44-45-46-47-48-49-50-51-52-53-54-55-56-57-58-59-60-61-62-63-64-65-66-67-68-69-70-71-72-73-74-75-76-77-78-79-80-81-82-83-84-85-86-87-88-89-90-91-92-93-94-95-96-97-98-99-100-101-102-103-104-105-106-107-108-109-110-111-112-113-114-115-116-117-118-119-120-121-122-

In [21]:
#g,g_c_i,y_hat,y_c,iter_i,ed_i
df_l = []
c_cg_found = []
i = 0
for info_i in new_info_back:
    g, g_c_i, y, y_hat, y_c, iter_i, ed_i = info_i
    # is it a counterfactual?
    df_l.append([i,"All",y_hat,iter_i,ed_i])
    if y_hat!=y_c:
        v_c = True
        df_l.append([i,"C-found",y_hat,iter_i,ed_i])
        c_cg_found.append((i,g,g_c_i,y_hat,"RCLI+BW"))
    else:
        v_c = False
    # is it well-classified?
    if y_hat==y:
        c_ok = "Good"
    else:
        c_ok = "Miss"
    df_l.append([i,c_ok,y_hat,iter_i,ed_i])
    i += 1

In [22]:
COUNT_SEARCH = "RCLI+BW"
df = pd.DataFrame(df_l, columns=['g_id','Status','Predicted Class','Iter','Density'])
COUNT_SEARCH = "dcg_{}_{}_RCLI+BW".format(l_search_type, h_search_type)
df.to_pickle("results/"+"{}_{}_{}.pickle".format(data_name,method,COUNT_SEARCH))

In [23]:
df = pd.DataFrame(c_cg_found, columns=['gid','G','C','y','Method'])
df.to_pickle("CG/"+"{}_{}_{}_cg.pickle".format(data_name,method,COUNT_SEARCH))